# Topic Modeling and sentiment analaysis

## Topic modeling

Topic Models, in a nutshell, are a type of statistical language models used for uncovering hidden structure in a collection of texts. In a practical and more intuitively, you can think of it as a task of:

Dimensionality Reduction, where rather than representing a text T in its feature space as {Word_i: count(Word_i, T) for Word_i in Vocabulary}, you can represent it in a topic space as {Topic_i: Weight(Topic_i, T) for Topic_i in Topics}

## Sentiment analysis

Sentiment analysis
It is used in social media monitoring, allowing businesses to gain insights about how customers feel about certain topics, and detect urgent issues in real time before they spiral out of control.

Our task here is to classify a tweet as a positive or negative tweet sentiment wise.



There are several existing algorithms we can use to perform the topic modeling. The most common of it are, Latent Semantic Analysis (LSA/LSI), Probabilistic Latent Semantic Analysis (pLSA), and Latent Dirichlet Allocation (LDA)


In this article, we’ll take a closer look at LDA, and implement our first topic model using the sklearn implementation in python.

# LDA Implementation

The steps used to implement LDA :
   1) Loading data
   
   2) Data cleaning
   
   
   3) Exploratory analysis
   
   
   4) Preparing data for LDA analysis
   
   
   5) LDA model training
   
   
   6) Analyzing LDA model results